# Data Extraction - PDF to Markdown

**Notebook ID:** `01_data_extraction`
**Description:** Convert PDF documents to Markdown format while preserving images and structure

---


## 📋 How to Use This Notebook

This notebook is **designed to work standalone** while maintaining state from previous notebooks.

### ✅ Prerequisites Check
- The first code cell checks for required files from previous notebooks
- If files are missing, you'll see a warning with instructions
- You can still run the notebook, but some cells may fail

### 🚀 Starting from This Notebook
If you want to start from this notebook (e.g., skip previous steps):

1. **Ensure prerequisite files exist** in the expected locations
2. **Update configuration** in the first code cell (PDF_NAME, paths, etc.)
3. **Run all cells** - the notebook will load existing state or create new outputs

### 💾 State Management
- Each notebook saves its state to `output/{notebook_name}_state.json`
- State includes file paths, configuration, and intermediate results
- Subsequent notebooks can load this state automatically

### 📁 File Locations
- **Output files**: Saved to `output/` directory
- **State files**: `output/{notebook_name}_state.json`
- **Generated data**: Check the `PROJECT_STATE` dictionary after running cells

### ⏭️ Next Notebook
After completing this notebook:
- Output files are automatically saved
- State is preserved for the next notebook
- Run the next notebook in sequence, or jump to any later notebook

---



In [ ]:
# CELL_ID: 01_data_extraction_dependency_check
# ============================================================================
# PREREQUISITE CHECK AND CONFIGURATION
# ============================================================================
# This cell checks for required files from previous notebooks and sets up
# the environment. If you're starting from this notebook, make sure prerequisite
# files exist, or run previous notebooks first.

from pathlib import Path
import json
import os

# ============================================================================
# CONFIGURATION: Set your project settings here
# ============================================================================
PDF_NAME = "Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1"  # Change if using different PDF
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Dictionary to store resolved file paths
PROJECT_STATE = {
    'pdf_name': PDF_NAME,
    'output_dir': str(OUTPUT_DIR),
}

# ============================================================================
# PREREQUISITE CHECK
# ============================================================================
missing_files = []
found_files = []

print("[OK] No prerequisites required for this notebook.\n")

# ============================================================================


In [ ]:
# CELL_ID: 01_data_extraction_dependency_check
# ============================================================================
# PREREQUISITE CHECK AND CONFIGURATION
# ============================================================================
# This cell checks for required files from previous notebooks and sets up
# the environment. If you're starting from this notebook, make sure prerequisite
# files exist, or run previous notebooks first.

from pathlib import Path
import json
import os

# ============================================================================
# CONFIGURATION: Set your project settings here
# ============================================================================
PDF_NAME = "Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1"  # Change if using different PDF
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

# Dictionary to store resolved file paths
PROJECT_STATE = {
    'pdf_name': PDF_NAME,
    'output_dir': str(OUTPUT_DIR),
}

# ============================================================================
# PREREQUISITE CHECK
# ============================================================================
missing_files = []
found_files = []

print("[OK] No prerequisites required for this notebook.\n")

# ============================================================================


## 1.1 PDF to Markdown Conversion with Images

This section converts the PDF to Markdown format while preserving images in their correct positions using Docling and PyMuPDF.


In [ ]:
# CELL_ID: 01_data_extraction_26252ff6
# Import required libraries
import json  # For parsing Docling's JSON structure
import os  # For operating system interface (though we use Path instead)
from pathlib import Path  # For cross-platform path handling
from docling.document_converter import DocumentConverter  # Docling: PDF to structured format converter
import fitz  # PyMuPDF: For precise image extraction from PDF pages

def extract_pdf_with_images(pdf_path, output_dir="output"):
    """
    Extract text and images from PDF using Docling + PyMuPDF.
    Uses Docling's JSON structure to correctly place images in markdown.
    
    This function performs a 4-step process:
    1. Convert PDF to Markdown and JSON using Docling
    2. Analyze JSON structure to determine image order in document
    3. Extract images precisely using PyMuPDF based on coordinates
    4. Inject extracted images into markdown at correct positions
    
    Args:
        pdf_path: Path to input PDF file
        output_dir: Directory where output files will be saved
    
    Returns:
        Dictionary with paths to output files and extraction statistics
    """
    # ============================================================================
    # SETUP: Create output directories
    # ============================================================================
    # Create main output directory if it doesn't exist (exist_ok=True prevents errors)
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # Create subdirectory for extracted images
    images_dir = output_path / "images"
    images_dir.mkdir(exist_ok=True)
    
    # Extract PDF filename without extension (e.g., "document.pdf" -> "document")
    pdf_name = Path(pdf_path).stem
    
    # ============================================================================
    # STEP 1: Convert PDF with Docling
    # ============================================================================
    print("Converting PDF with Docling...")
    # Initialize Docling converter (handles OCR, layout analysis, structure detection)
    converter = DocumentConverter()
    # Convert PDF - this analyzes the document structure, extracts text, identifies images
    result = converter.convert(pdf_path)
    
    # Export the converted document to Markdown format
    # This gives us the text content with placeholders for images
    markdown_content = result.document.export_to_markdown()
    
    # Save raw markdown (without images) to file
    markdown_path = output_path / f"{pdf_name}.md"
    with open(markdown_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
    print(f"Markdown saved to: {markdown_path}")
    
    # Export the full document structure as JSON
    # The JSON contains detailed information: text blocks, images, tables, their positions, hierarchy
    json_content = result.document.export_to_dict()
    
    # Save JSON structure for reference and analysis
    json_path = output_path / f"{pdf_name}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_content, f, indent=2)
    print(f"JSON saved to: {json_path}")
    
    # ============================================================================
    # STEP 2: Parse JSON structure to determine image order
    # ============================================================================
    print("\nAnalyzing document structure...")
    # The 'body' contains the main document content
    body = json_content.get("body", {})
    # 'children' is an ordered list of all document elements (texts, images, tables)
    # This order reflects the actual reading order in the PDF
    children = body.get("children", [])
    
    # Get all picture objects from the JSON
    pictures = json_content.get("pictures", [])
    
    # Create a mapping: picture self-reference -> picture index
    # Each picture has a unique "self_ref" like "#/pictures/0", "#/pictures/1", etc.
    # We need to map these references to their array indices
    picture_ref_to_index = {}
    for idx, picture in enumerate(pictures):
        self_ref = picture.get("self_ref", "")
        if self_ref:
            picture_ref_to_index[self_ref] = idx
    
    # Build ordered list of picture indices based on document reading order
    # We iterate through body.children to find picture references in the order they appear
    picture_order = []
    for child in children:
        ref = child.get("$ref", "")  # Get the reference (e.g., "#/pictures/5")
        if ref and "#/pictures/" in ref:  # Check if this is a picture reference
            picture_idx = picture_ref_to_index.get(ref, None)
            if picture_idx is not None:
                picture_order.append(picture_idx)  # Add to ordered list
    
    print(f"Found {len(pictures)} pictures in JSON")
    print(f"Found {len(picture_order)} picture references in document structure")
    
    # If no pictures found, return early with basic info
    if len(pictures) == 0:
        print("No pictures found in JSON. Check if PDF contains images.")
        return {
            "markdown_path": str(markdown_path),
            "json_path": str(json_path),
            "images_dir": str(images_dir),
            "num_pictures": 0,
            "num_images_extracted": 0
        }
    
    # ============================================================================
    # STEP 3: Extract images using PyMuPDF (precise coordinate-based extraction)
    # ============================================================================
    print("\nExtracting images with PyMuPDF...")
    # Open the original PDF with PyMuPDF for precise image extraction
    pdf_doc = fitz.open(pdf_path)
    
    # Dictionary to store image information: picture_index -> {path, caption, page, etc.}
    image_mapping = {}
    
    # Process pictures in the order they appear in the document
    for picture_idx in picture_order:
        if picture_idx >= len(pictures):
            continue  # Skip if index is out of bounds
            
        picture = pictures[picture_idx]
        try:
            # Get provenance information - tells us where the image is located
            prov = picture.get("prov", [{}])[0]  # prov is a list, get first element
            page_no = prov.get("page_no", 1)  # Page number (1-indexed in Docling)
            bbox = prov.get("bbox", {})  # Bounding box: {l: left, t: top, r: right, b: bottom}
            coord_origin = prov.get("coord_origin", "BOTTOMLEFT")  # Coordinate system origin
            
            # Get the specific PDF page (PyMuPDF uses 0-indexed pages)
            page = pdf_doc[page_no - 1]
            page_height = page.rect.height  # Get page height for coordinate conversion
            
            # Convert coordinates from Docling's BOTTOMLEFT system to PyMuPDF's TOPLEFT system
            # Docling: (0,0) is bottom-left, y increases upward
            # PyMuPDF: (0,0) is top-left, y increases downward
            if coord_origin == "BOTTOMLEFT":
                x0 = bbox["l"]  # Left coordinate (same in both systems)
                y0 = page_height - bbox["t"]  # Convert top: flip vertically
                x1 = bbox["r"]  # Right coordinate
                y1 = page_height - bbox["b"]  # Convert bottom: flip vertically
            else:
                # Already in TOPLEFT format
                x0 = bbox["l"]
                y0 = bbox["t"]
                x1 = bbox["r"]
                y1 = bbox["b"]
            
            # Create a rectangle defining the image area to extract
            rect = fitz.Rect(x0, y0, x1, y1)
            
            # Render the page region as a pixmap (raster image)
            # Matrix(2, 2) = 2x zoom for better image quality
            # clip=rect extracts only the specified rectangle
            mat = fitz.Matrix(2, 2)
            pix = page.get_pixmap(matrix=mat, clip=rect)
            
            # Generate filename based on picture index and page number
            img_filename = f"picture_{picture_idx:03d}_page_{page_no}.png"
            img_path = images_dir / img_filename
            pix.save(str(img_path))  # Save as PNG image
            
            # ====================================================================
            # Extract caption for the image
            # ====================================================================
            caption = ""
            
            # Method 1: Check if picture has explicit captions in JSON
            if picture.get("captions"):
                caption = picture["captions"][0].get("text", "").strip()
            
            # Method 2: Look for nearby text elements that might be captions
            # (e.g., "Figure 1: Description")
            if not caption:
                # Find this picture's position in the document structure
                for child_idx, child in enumerate(children):
                    if child.get("$ref") == picture.get("self_ref"):
                        # Look at nearby text elements (before and after the image)
                        texts = json_content.get("texts", [])
                        # Check 1-2 positions ahead and behind
                        for offset in [1, 2, -1, -2]:
                            check_idx = child_idx + offset
                            if 0 <= check_idx < len(children):
                                text_ref = children[check_idx].get("$ref", "")
                                if "#/texts/" in text_ref:
                                    text_idx = int(text_ref.split("/")[-1])
                                    if text_idx < len(texts):
                                        text_content = texts[text_idx].get("text", "").strip()
                                        # Heuristic: captions usually start with "Figure" or are short descriptions
                                        if text_content.startswith("Figure") or (len(text_content) < 200 and len(text_content) > 5):
                                            caption = text_content
                                            break
                        break
            
            # Fallback: use generic caption if none found
            if not caption:
                caption = f"Figure {picture_idx + 1}"
            
            # Store image information for later insertion into markdown
            relative_path = f"images/{img_filename}"  # Relative path for markdown
            image_mapping[picture_idx] = {
                "index": picture_idx,
                "page": page_no,
                "caption": caption,
                "path": relative_path
            }
            
            print(f"  [{len(image_mapping)}/{len(picture_order)}] Saved: {img_filename} (page {page_no}) - {caption[:60]}...")
            
        except Exception as e:
            print(f"  Error extracting picture {picture_idx}: {e}")
            import traceback
            traceback.print_exc()
    
    pdf_doc.close()  # Close the PDF file
    
    # ============================================================================
    # STEP 4: Inject images into Markdown at correct positions
    # ============================================================================
    print(f"\nInjecting {len(image_mapping)} images into Markdown...")
    # Split markdown into lines for processing
    markdown_lines = markdown_content.split("\n")
    new_markdown_lines = []  # Will contain the final markdown with images
    
    # Counter to track which image we're inserting (maintains order)
    image_insertion_counter = 0
    
    # Process each line of the markdown
    for line_idx, line in enumerate(markdown_lines):
        # Docling places "<!-- image -->" comments where images should appear
        if "<!-- image -->" in line:
            # Insert the next image in document order
            if image_insertion_counter < len(picture_order):
                picture_idx = picture_order[image_insertion_counter]
                if picture_idx in image_mapping:
                    img_info = image_mapping[picture_idx]
                    # Replace placeholder with Markdown image syntax: ![alt text](path)
                    img_tag = f"![{img_info['caption']}]({img_info['path']})"
                    new_line = line.replace("<!-- image -->", img_tag)
                    new_markdown_lines.append(new_line)
                    image_insertion_counter += 1
                else:
                    # Image extraction failed, keep placeholder but mark it
                    new_markdown_lines.append(line.replace("<!-- image -->", "<!-- image (not extracted) -->"))
                    image_insertion_counter += 1
            else:
                # More placeholders than images (shouldn't happen, but handle gracefully)
                new_markdown_lines.append(line)
        else:
            # Regular line, keep as-is
            new_markdown_lines.append(line)
    
    # ============================================================================
    # SAVE RESULTS
    # ============================================================================
    # Join lines back into full markdown document
    enhanced_markdown = "\n".join(new_markdown_lines)
    enhanced_md_path = output_path / f"{pdf_name}_with_images.md"
    with open(enhanced_md_path, "w", encoding="utf-8") as f:
        f.write(enhanced_markdown)
    
    print(f"\n✓ Enhanced Markdown saved to: {enhanced_md_path}")
    print(f"✓ Total images extracted: {len(image_mapping)}")
    print(f"✓ Images inserted: {image_insertion_counter}")
    print(f"✓ Images directory: {images_dir}")
    
    # Return summary dictionary for programmatic access to results
    return {
        "markdown_path": str(markdown_path),
        "enhanced_markdown_path": str(enhanced_md_path),
        "json_path": str(json_path),
        "images_dir": str(images_dir),
        "num_pictures": len(pictures),
        "num_images_extracted": len(image_mapping),
        "num_images_inserted": image_insertion_counter
    }




In [ ]:
# CELL_ID: 01_data_extraction_ce45d194
# Separate execution cell - run the conversion
pdf_file = "Kenya-National-Clinical-Guidelines-for-the-Management-of-Diabetes-2nd-Editiion-2018-1.pdf"
results = extract_pdf_with_images(pdf_file, output_dir="output")

# Display results summary
print("\n" + "="*60)
print("CONVERSION SUMMARY")
print("="*60)
print(f"✓ Markdown (raw): {results['markdown_path']}")
print(f"✓ Markdown (with images): {results['enhanced_markdown_path']}")
print(f"✓ JSON structure: {results['json_path']}")
print(f"✓ Images directory: {results['images_dir']}")
print(f"\n📊 Statistics:")
print(f"  • Total pictures found: {results['num_pictures']}")
print(f"  • Images extracted: {results['num_images_extracted']}")
print(f"  • Images inserted: {results['num_images_inserted']}")
print("="*60)